In [ ]:
import numpy as np
import pandas as pd

wine = pd.read_csv('data/Aussie_wines.csv',index_col=0)
wine.head()

In [ ]:
wine['province'].value_counts(dropna=False)

In [ ]:
import pandas as pd
from bokeh.plotting import figure
from bokeh.layouts import layout, widgetbox
from bokeh.models import ColumnDataSource, HoverTool, BoxZoomTool, ResetTool, PanTool
from bokeh.models.widgets import Slider, Select, TextInput, Div
from bokeh.models import WheelZoomTool, SaveTool, LassoSelectTool
from bokeh.io import curdoc
from functools import lru_cache

In [ ]:
@lru_cache()
def load_data():
    df = pd.read_csv("data/Aussie_wines.csv", index_col=0)
    return df

In [ ]:
cols = ["price", "points", "variety", "province", "description"]

states = ["All", "South Australia", "Victoria", "Western Australia", "Australia Other", "New South Wales", "Tasmania"]

In [ ]:
desc = Div(text="All Provinces", width=800)
province = Select(title="Province", options=states, value="All")
price_max = Slider(start=0, end=900, step=5, value=200, title="Maximum Price")
title = TextInput(title="Title Contains")
details = Div(text="Selection Details:", width=800)

In [ ]:
source = ColumnDataSource(data=load_data())

In [ ]:
hover = HoverTool(tooltips=[
    ("title", "@title"),
    ("variety", "@variety"),
])
TOOLS = [
    hover, BoxZoomTool(), LassoSelectTool(), WheelZoomTool(), PanTool(),
    ResetTool(), SaveTool()
]

In [ ]:
p = figure(
    plot_height=600,
    plot_width=700,
    title="Australian Wine Analysis",
    tools=TOOLS,
    x_axis_label="points",
    y_axis_label="price (USD)",
    toolbar_location="above")

p.circle(
    y="price",
    x="points",
    source=source,
    color="variety_color",
    size=7,
    alpha=0.4)

In [ ]:
def select_reviews():
    """ Use the current selections to determine which filters to apply to the
    data. Return a dataframe of the selected data
    """
    df = load_data()

    # Determine what has been selected for each widgetd
    max_price = price_max.value
    province_val = province.value
    title_val = title.value

    # Filter by price and province
    if province_val == "All":
        selected = df[df.price <= max_price]
    else:
        selected = df[(df.province == province_val) & (df.price <= max_price)]

    # Further filter by string in title if it is provided
    if title_val != "":
        selected = selected[selected.title.str.contains(title_val) == True]

    # Example showing how to update the description
    desc.text = "Province: {} and Price < {}".format(province_val, max_price)
    return selected


In [ ]:
def update():
    """ Get the selected data and update the data in the source
    """
    df_active = select_reviews()
    source.data = ColumnDataSource(data=df_active).data


def selection_change(attrname, old, new):
    """ Function will be called when the poly select (or other selection tool)
    is used. Determine which items are selected and show the details below
    the graph
    """
    selected = source.selected["1d"]["indices"]

    # Need to get a list of the active reviews so the indices will match up
    df_active = select_reviews()

    # If something is selected, then get those details and format the results
    # as an HTML table
    if selected:
        data = df_active.iloc[selected, :]
        temp = data.set_index("title").T.reindex(index=col_order)
        details.text = temp.style.render()
    else:
        details.text = "Selection Details"

In [ ]:
source.on_change("selected", selection_change)

In [ ]:
inputs = widgetbox(*controls, sizing_mode="fixed")


In [ ]:
l = layout([[desc], [inputs, p], [details]], sizing_mode="fixed")
update()
curdoc().add_root(l)

# Show the title in the browser bar
curdoc().title = "Australian Wine Analysis"